### Analyze the data

In the blog post I'm following it outputs the following information: 

```
1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"
```

I want to add how many phrases I hae but to generally create this output here

In [31]:
import pandas as pd 
from tools import load_data

In [32]:
data = load_data()

In [33]:
data.head()

,english,french,french_tokens,english_tokens,english_bow,french_bow,eng_len,fren_len,english_padded,french_padded
0,"new jersey is sometimes quiet during autumn , ...",new jersey est parfois calme pendant l' automn...,"[new, jersey, est, parfois, calme, pendant, l,...","[new, jersey, is, sometimes, quiet, during, au...","[8, 7, 5, 11, 9, 3, 2, 0, 6, 5, 10, 4, 1]","[10, 7, 4, 11, 2, 12, 8, 0, 5, 6, 4, 9, 3, 1]",13,14,"[8, 7, 5, 11, 9, 3, 2, 0, 6, 5, 10, 4, 1, -1, ...","[10, 7, 4, 11, 2, 12, 8, 0, 5, 6, 4, 9, 3, 1, ..."
1,the united states is usually chilly during jul...,les états-unis est généralement froid en juill...,"[les, états, unis, est, généralement, froid, e...","[the, united, states, is, usually, chilly, dur...","[17, 18, 16, 5, 19, 12, 3, 14, 0, 6, 5, 19, 13...","[18, 21, 20, 4, 15, 13, 3, 17, 5, 6, 14, 16, 3...",15,14,"[17, 18, 16, 5, 19, 12, 3, 14, 0, 6, 5, 19, 13...","[18, 21, 20, 4, 15, 13, 3, 17, 5, 6, 14, 16, 3..."
2,"california is usually quiet during march , and...","california est généralement calme en mars , et...","[california, est, généralement, calme, en, mar...","[california, is, usually, quiet, during, march...","[20, 5, 19, 9, 3, 23, 0, 6, 5, 19, 21, 4, 22]","[22, 4, 15, 2, 3, 25, 5, 6, 4, 15, 23, 3, 24]",13,13,"[20, 5, 19, 9, 3, 23, 0, 6, 5, 19, 21, 4, 22, ...","[22, 4, 15, 2, 3, 25, 5, 6, 4, 15, 23, 3, 24, ..."
3,the united states is sometimes mild during jun...,"les états-unis est parfois légère en juin , et...","[les, états, unis, est, parfois, légère, en, j...","[the, united, states, is, sometimes, mild, dur...","[17, 18, 16, 5, 11, 25, 3, 22, 0, 6, 5, 24, 4,...","[18, 21, 20, 4, 11, 27, 3, 24, 5, 6, 26, 13, 3...",14,14,"[17, 18, 16, 5, 11, 25, 3, 22, 0, 6, 5, 24, 4,...","[18, 21, 20, 4, 11, 27, 3, 24, 5, 6, 26, 13, 3..."
4,"your least liked fruit is the grape , but my l...","votre moins aimé fruit est le raisin , mais mo...","[votre, moins, aimé, fruit, est, le, raisin, m...","[your, least, liked, fruit, is, the, grape, bu...","[34, 31, 32, 29, 5, 17, 30, 28, 33, 31, 32, 5,...","[38, 34, 29, 30, 4, 32, 37, 33, 35, 34, 29, 4,...",14,14,"[34, 31, 32, 29, 5, 17, 30, 28, 33, 31, 32, 5,...","[38, 34, 29, 30, 4, 32, 37, 33, 35, 34, 29, 4,..."


In [29]:
type(data['french_bow'][0][0])

str

In [4]:
from collections import Counter

In [15]:
tokens = data['english_tokens'].apply(lambda x: x.strip("['']").split("', '"))

In [18]:
tokens[0]

['new',
 'jersey',
 'is',
 'sometimes',
 'quiet',
 'during',
 'autumn',
 'and',
 'it',
 'is',
 'snowy',
 'in',
 'april']